<a href="https://colab.research.google.com/github/Roy1127/c4/blob/master/CIFAR_10_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Imports


In [1]:
import numpy as np
import time

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
def train_cifar10(model, dataloader, n_epochs=1, optimizer=None,
          lr=0.001, weight_decay=0.001, loss_fn=None, isPytorch=True):
    np.random.seed(4)
    opt = optimizer(model.parameters(), lr = lr, weight_decay = weight_decay)

    for epoch in range(n_epochs):
        start = time.time()
        avg_acc, avg_loss = 0, 0
        if isPytorch:
          avg_acc, avg_loss = epoch_pytorch_cifar10(dataloader, model, loss_fn=loss_fn, opt=opt)
        else:
          avg_acc, avg_loss = epoch_general_cifar10(dataloader, model, loss_fn=loss_fn, opt=opt)
        epoch_t = time.time() - start
        print(f"Epoch: {epoch}, Acc: {avg_acc}, Loss: {avg_loss}, Time: {epoch_t:.2f}s")

    return avg_acc, avg_loss

def epoch_pytorch_cifar10(dataloader, model, loss_fn, opt=None):
    train_loss = 0.0
    correct = 0.0

    # training model loop
    model.train()
    for data, labels in dataloader:

      if train_on_gpu:
        data, labels = data.cuda(), labels.cuda()

      opt.zero_grad()
      output = model(data)
      loss = loss_fn(output, labels)
      loss.backward()
      opt.step()
      train_loss += loss.item() * data.size(0)
      _, predicted = torch.max(output, 1)
      correct += (predicted == labels).sum().item()

    return correct/len(dataloader.dataset), train_loss/len(dataloader.dataset)

def epoch_general_cifar10(dataloader, model, loss_fn=ndl.nn.SoftmaxLoss(), opt=None):
    np.random.seed(4)
    correct, total_loss = 0, 0

    if opt is None:
        model.eval()
        for batch in dataloader:
            X, y = batch
            X, y = ndl.Tensor(X, device=device), ndl.Tensor(y, device=device)
            out = model(X)
            loss = loss_fn(out, y)
            correct += np.sum(np.argmax(out.numpy(), axis=1) == y.numpy())
            total_loss += loss.data.numpy() * y.shape[0]
    else:
        model.train()
        for batch in dataloader:
            opt.reset_grad()
            X, y = batch
            X, y = ndl.Tensor(X, device=device), ndl.Tensor(y, device=device)
            out = model(X)
            loss = loss_fn(out, y)
            loss.backward()
            opt.step()
            correct += np.sum(np.argmax(out.numpy(), axis=1) == y.numpy())
            total_loss += loss.data.numpy() * y.shape[0]

    sample_nums = len(dataloader.dataset)
    return correct / sample_nums, total_loss / sample_nums

####Check for GPU

In [ ]:
print('PyTorch version:', torch.__version__)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

PyTorch version: 2.5.0+cu121
Sun Nov  3 10:43:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+--------------------------------------

In [ ]:
# Set random seed for reproducability
torch.manual_seed(271828)
np.random.seed(271728)

##PyTorch Model for Cifar10

####Data Preparation for PyTorch

In [ ]:
# transform for the training data
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.1307], [0.3081])
])

# load datasets, downloading if needed
train_set = CIFAR10('./data/cifar10', train=True, download=True,
                  transform=train_transform)

train_len = len(train_set)
idxes = list(range(train_len))

#shuffle the indexes
np.random.shuffle(idxes)

split = int(np.floor(0.2*train_len))
train_idx, valid_idx = idxes[split: ], idxes[:split]

#Sampler gets the batches of data
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_len = len(train_set)
idxes = list(range(train_len))

#shuffle the indexes
np.random.shuffle(idxes)

split = int(np.floor(0.2*train_len))
train_idx, valid_idx = idxes[split: ], idxes[:split]

#Sampler gets the batches of data
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=20, sampler=train_sampler, num_workers=0)
valid_loader = torch.utils.data.DataLoader(train_set, batch_size=20, sampler=valid_sampler, num_workers=0)

100%|██████████| 170M/170M [00:02<00:00, 76.9MB/s]


Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10
Files already downloaded and verified


###Model

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.conv1 = conv_block(in_channels, 16)
        self.conv2 = conv_block(16, 32, pool=True)
        self.res1 = nn.Sequential(conv_block(32, 32), conv_block(32, 32))

        self.conv3 = conv_block(32, 64, pool=True)
        self.conv4 = conv_block(64, 128, pool=True)
        self.res2 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d((1,1)),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(128, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

model = ResNet9(3, 10)
print(model)

if train_on_gpu:
  model.cuda()

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

###Training

In [ ]:
train_cifar10(model, train_loader, n_epochs=10, optimizer=optim.Adam, lr=1e-4, loss_fn=nn.CrossEntropyLoss())

Epoch: 0, Acc: 0.41844, Loss: 1.101134807085991, Time: 39.75s
Epoch: 1, Acc: 0.5559, Loss: 0.6986673478364944, Time: 39.29s
Epoch: 2, Acc: 0.61686, Loss: 0.5276486627548933, Time: 39.44s
Epoch: 3, Acc: 0.65254, Loss: 0.4206232277840376, Time: 39.08s
Epoch: 4, Acc: 0.68348, Loss: 0.3356303848192096, Time: 39.39s
Epoch: 5, Acc: 0.70388, Loss: 0.2789133390113711, Time: 39.23s
Epoch: 6, Acc: 0.7238, Loss: 0.2222105843245983, Time: 39.37s
Epoch: 7, Acc: 0.73588, Loss: 0.18663425630070268, Time: 39.44s
Epoch: 8, Acc: 0.74752, Loss: 0.15300494082830846, Time: 39.38s
Epoch: 9, Acc: 0.75538, Loss: 0.1315854042828083, Time: 39.49s


(0.75538, 0.1315854042828083)

##Custom Model for Cifar10

In [2]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714
!git clone https://github.com/dlsys10714/hw4.git
%cd /content/drive/MyDrive/10714/hw4

!pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!pip3 install pybind11

%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd


Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714
fatal: destination path 'hw4' already exists and is not an empty directory.
/content/drive/MyDrive/10714/hw4
  Cloning https://github.com/dlsys10714/mugrade.git to /tmp/pip-req-build-gukw756v
  Running command git clone --filter=blob:none --quiet https://github.com/dlsys10714/mugrade.git /tmp/pip-req-build-gukw756v
  Resolved https://github.com/dlsys10714/mugrade.git to commit 656cdc2b7ad5a37e7a5347a7b0405df0acd72380
  Preparing metadata (setup.py) ... done
  Created wheel for mugrade: filename=mugrade-1.2-py3-none-any.whl size=3935 sha256=0bd6febf117fdd0d468597ccc8170f6610c83803f92a5b97bed8b1def0e347fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-5_xywv0j/wheels/8b/ba/3a/621da1207eab160c01968c5e0bd1266f505b9e3f8010376d61
Successfully built mugrade
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 4.0 MB/s eta 0:00:00
env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


### Data Preparation for Custom Framework

In [4]:
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
from models import ResNet9
from simple_ml import train_cifar10, evaluate_cifar10

dataset = ndl.data.CIFAR10Dataset("data/cifar-10-batches-py", train=True)
dataloader = ndl.data.DataLoader(\
         dataset=dataset,
         batch_size=128,
         shuffle=True)

###Model

In [5]:
class ResNet9(ndl.nn.Module):
    def __init__(self, device=None, dtype="float32"):
        super().__init__()
        ### BEGIN YOUR SOLUTION ###
        self.device = device
        self.dtype = dtype
        self.model = self.get_model()
        ### END YOUR SOLUTION

    def ConvBN(self, a, b, k, s):
        return nn.Sequential(
            nn.Conv(a, b, kernel_size = k, stride = s, device = self.device, dtype = self.dtype),
            nn.BatchNorm2d(b, device = self.device, dtype = self.dtype),
            nn.ReLU()
        )

    def get_model(self):
        return nn.Sequential(
            self.ConvBN(3, 16, 7, 4),
            self.ConvBN(16, 32, 3, 2),
            nn.Residual(
                nn.Sequential(
                    self.ConvBN(32, 32, 3, 1),
                    self.ConvBN(32, 32, 3, 1)
                )
            ),
            self.ConvBN(32, 64, 3, 2),
            self.ConvBN(64, 128, 3, 2),
            nn.Residual(
                nn.Sequential(
                    self.ConvBN(128, 128, 3, 1),
                    self.ConvBN(128, 128, 3, 1)
                )
            ),
            nn.Flatten(),
            nn.Linear(128, 128, device = self.device, dtype = self.dtype),
            nn.ReLU(),
            nn.Linear(128, 10, device = self.device, dtype = self.dtype)
        )

    def forward(self, x):
        ### BEGIN YOUR SOLUTION
        return self.model(x)
        ### END YOUR SOLUTION

device = ndl.cuda()
model = ResNet9(device=device, dtype="float32")

###Training

In [6]:
train_cifar10(model, dataloader, n_epochs=10, optimizer=ndl.optim.Adam,
      lr=0.001, weight_decay=0.001)

Optimizer: <needle.optim.Adam object at 0x7e21998dc4c0>, Loss function: <needle.nn.nn_basic.SoftmaxLoss object at 0x7e2198ea0fd0>
Epoch: 0, Acc: 0.38966, Loss: [1.6985745], Time: 76.25s
Optimizer: <needle.optim.Adam object at 0x7e21998dc4c0>, Loss function: <needle.nn.nn_basic.SoftmaxLoss object at 0x7e2198ea0fd0>
Epoch: 1, Acc: 0.49714, Loss: [1.3992902], Time: 76.25s
Optimizer: <needle.optim.Adam object at 0x7e21998dc4c0>, Loss function: <needle.nn.nn_basic.SoftmaxLoss object at 0x7e2198ea0fd0>
Epoch: 2, Acc: 0.54294, Loss: [1.2691967], Time: 76.48s
Optimizer: <needle.optim.Adam object at 0x7e21998dc4c0>, Loss function: <needle.nn.nn_basic.SoftmaxLoss object at 0x7e2198ea0fd0>
Epoch: 3, Acc: 0.57882, Loss: [1.1748431], Time: 76.53s
Optimizer: <needle.optim.Adam object at 0x7e21998dc4c0>, Loss function: <needle.nn.nn_basic.SoftmaxLoss object at 0x7e2198ea0fd0>
Epoch: 4, Acc: 0.60848, Loss: [1.099906], Time: 76.76s
Optimizer: <needle.optim.Adam object at 0x7e21998dc4c0>, Loss function:

(0.69814, array([0.85183305], dtype=float32))